## Mochammad Daffa Putra Karyudi

## Import Libraries

In [1]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.express as px
from keras import layers, models
import plotly.graph_objects as go
from keras.regularizers import l2
from plotly.subplots import make_subplots
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc


2024-12-02 10:27:08.513631: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-02 10:27:08.526668: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-02 10:27:08.530460: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 10:27:08.539403: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Set Display Options

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Level 1
The Data Kingdom’s Entry Gate – Understanding the Terrain

## Data Overview

In [ ]:
df = pd.read_csv('./Customer - Telco Data.csv')

### Basic Statistics

In [4]:
print("Dataset Info:")
print(df.info())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-n

In [5]:
print("Basic Statistics:")
print(df.describe())

Basic Statistics:
       SeniorCitizen       tenure  MonthlyCharges
count    7043.000000  7043.000000     7043.000000
mean        0.162147    32.371149       64.761692
std         0.368612    24.559481       30.090047
min         0.000000     0.000000       18.250000
25%         0.000000     9.000000       35.500000
50%         0.000000    29.000000       70.350000
75%         0.000000    55.000000       89.850000
max         1.000000    72.000000      118.750000


In [6]:
print("Missing Values:")
print(df.isnull().sum())

Missing Values:
customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


In [7]:
print("Duplicated ID:")
print(df.duplicated(subset='customerID').sum())

Duplicated ID:
0


### Data Exploration

In [8]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [9]:
print("Unique values in categorical columns:")
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print("="*10)
    print(f"\n{col}:")
    print(df[col].value_counts())

Unique values in categorical columns:

customerID:
customerID
7590-VHVEG    1
3791-LGQCY    1
6008-NAIXK    1
5956-YHHRX    1
5365-LLFYV    1
             ..
9796-MVYXX    1
2637-FKFSY    1
1552-AAGRX    1
4304-TSPVK    1
3186-AJIEK    1
Name: count, Length: 7043, dtype: int64

gender:
gender
Male      3555
Female    3488
Name: count, dtype: int64

Partner:
Partner
No     3641
Yes    3402
Name: count, dtype: int64

Dependents:
Dependents
No     4933
Yes    2110
Name: count, dtype: int64

PhoneService:
PhoneService
Yes    6361
No      682
Name: count, dtype: int64

MultipleLines:
MultipleLines
No                  3390
Yes                 2971
No phone service     682
Name: count, dtype: int64

InternetService:
InternetService
Fiber optic    3096
DSL            2421
No             1526
Name: count, dtype: int64

OnlineSecurity:
OnlineSecurity
No                     3498
Yes                    2019
No internet service    1526
Name: count, dtype: int64

OnlineBackup:
OnlineBackup
No       

# Level 2
Power-Ups & Special Abilities – Finding the Drivers of Churn

## Data Preprocessing

In [10]:
df_encoded = df.copy()

### Label Encoding

#### Encode

In [11]:
# Separate binary (Yes/No) columns from other categorical columns
binary_columns = ['Churn', 'PhoneService', 'PaperlessBilling', 'Partner', 'Dependents']
other_categorical = ['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod', 'gender', 
                    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                    'TechSupport', 'StreamingTV', 'StreamingMovies']

# Create a copy of the dataframe
df_encoded = df.copy()

# Manual mapping for binary columns (Yes/No)
binary_map = {'No': 0, 'Yes': 1}
for col in binary_columns:
    df_encoded[col] = df_encoded[col].map(binary_map)

# Use LabelEncoder for other categorical columns
le = LabelEncoder()
for col in other_categorical:
    df_encoded[col] = le.fit_transform(df_encoded[col])

print("Shape of encoded data:", df_encoded.shape)
print("\nFirst few rows of encoded data:")
display(df_encoded.head(10))

# Display mapping for other categorical columns
print("\nMapping for other categorical columns:")
for col in other_categorical:
    unique_values = df[col].unique()
    encoded_values = le.fit_transform(unique_values)
    mapping = dict(zip(unique_values, encoded_values))
    print(f"\n{col}:")
    for original, encoded in mapping.items():
        print(f"{original} -> {encoded}")

Shape of encoded data: (7043, 21)

First few rows of encoded data:


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,0
1,5575-GNVDE,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.5,0
2,3668-QPYBK,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,1
3,7795-CFOCW,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1840.75,0
4,9237-HQITU,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,151.65,1
5,9305-CDSKC,0,0,0,0,8,1,2,1,0,0,2,0,2,2,0,1,2,99.65,820.5,1
6,1452-KIOVK,1,0,0,1,22,1,2,1,0,2,0,0,2,0,0,1,1,89.10,1949.4,0
7,6713-OKOMC,0,0,0,0,10,0,1,0,2,0,0,0,0,0,0,0,3,29.75,301.9,0
8,7892-POOKP,0,0,1,0,28,1,2,1,0,0,2,2,2,2,0,1,2,104.80,3046.05,1
9,6388-TABGU,1,0,0,1,62,1,0,0,2,2,0,0,0,0,1,0,0,56.15,3487.95,0



Mapping for other categorical columns:

MultipleLines:
No phone service -> 1
No -> 0
Yes -> 2

InternetService:
DSL -> 0
Fiber optic -> 1
No -> 2

Contract:
Month-to-month -> 0
One year -> 1
Two year -> 2

PaymentMethod:
Electronic check -> 2
Mailed check -> 3
Bank transfer (automatic) -> 0
Credit card (automatic) -> 1

gender:
Female -> 0
Male -> 1

OnlineSecurity:
No -> 0
Yes -> 2
No internet service -> 1

OnlineBackup:
Yes -> 2
No -> 0
No internet service -> 1

DeviceProtection:
No -> 0
Yes -> 2
No internet service -> 1

TechSupport:
No -> 0
Yes -> 2
No internet service -> 1

StreamingTV:
No -> 0
Yes -> 2
No internet service -> 1

StreamingMovies:
No -> 0
Yes -> 2
No internet service -> 1


#### Decode

In [12]:
# Create a copy of the encoded dataframe
df_decoded = df_encoded.copy()

# Decode binary columns using reverse mapping
binary_map_reverse = {0: 'No', 1: 'Yes'}
for col in binary_columns:
    df_decoded[col] = df_decoded[col].map(binary_map_reverse)

# Create dictionary to store LabelEncoder objects for other categorical columns
label_encoders = {}

# Fit and store LabelEncoder for each non-binary categorical column
for col in other_categorical:
    le = LabelEncoder()
    label_encoders[col] = le.fit(df[col])  # Fit using original data
    df_decoded[col] = label_encoders[col].inverse_transform(df_encoded[col])

print("First few rows of decoded data:")
display(df_decoded.head(10))

# Verify decoding by comparing with original data
print("\nVerification - First few rows of original data:")
display(df.head(10))

# Verify that all values match the original
binary_match = all((df_decoded[col] == df[col]).all() for col in binary_columns)
categorical_match = all((df_decoded[col] == df[col]).all() for col in other_categorical)

print("\nAll binary columns matched:", binary_match)
print("All other categorical columns matched:", categorical_match)

First few rows of decoded data:


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,Yes,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No



Verification - First few rows of original data:


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,Yes,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No



All binary columns matched: True
All other categorical columns matched: True


## Visualization

### Correlation Heatmap

In [13]:
# Get only numeric columns
numeric_columns = df_encoded.select_dtypes(include=['int64', 'float64']).columns
correlation_matrix = df_encoded[numeric_columns].corr(method='pearson')

# Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    text=correlation_matrix.round(2),
    texttemplate='%{text}',
    textfont={"size": 10},
    hoverongaps=False,
    colorscale='RdBu',
    zmid=0,
    showscale=True
))

# Update layout
fig.update_layout(
    title={
        'text': 'Correlation Matrix of Numeric Features',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    width=1400,
    height=1000,
    xaxis={
        'tickangle': 45,
        'tickfont': {'size': 10}
    },
    yaxis={
        'tickangle': 0,
        'tickfont': {'size': 10}
    },
    margin=dict(t=100, l=100, r=100, b=100)  # Adjust margins
)

# Add hover template
fig.update_traces(
    hovertemplate="X: %{x}<br>Y: %{y}<br>Correlation: %{text}<extra></extra>"
)

# Display
fig.show()

#### Churn Correlation

In [14]:
# Sort correlation values
churn_correlation = df_encoded[numeric_columns].corr(method='pearson')['Churn'].drop('Churn')
churn_correlation = churn_correlation.sort_values()

# Determine colors based on correlation
colors = ['red' if x < 0 else 'green' for x in churn_correlation.values]

# Create the diverging bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    y=churn_correlation.index,
    x=churn_correlation.values,
    orientation='h',
    marker_color=colors,
    marker_line_color='white',
    marker_line_width=0.5,
    opacity=0.7
))

fig.update_layout(
    title={
        'text': 'Feature Correlation with Churn',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 20}
    },
    xaxis_title='Correlation Coefficient',
    yaxis_title='Features',
    width=1000,
    height=800,
    xaxis=dict(
        zeroline=True,
        zerolinewidth=2,
        zerolinecolor='black',
        gridcolor='lightgrey',
        griddash='dash',
        range=[-1, 1],
        dtick=0.2
    ),
    yaxis=dict(
        tickfont=dict(size=12)
    ),
    plot_bgcolor='white'
)

# Add a vertical line at x=0
fig.add_vline(x=0, line_width=1, line_color="black")

# Display
fig.show()

#### Analysis
1. Primary Churn Risk Factors:
    - Contract Type is the strongest protective factor (-0.397):
        - Month-to-month customers are more likely to churn
        - Long-term contracts significantly reduce churn risk
    - Customer Tenure has strong negative correlation (-0.352):
        - New customers are at higher risk of churning
        - Loyalty increases with tenure

2. Service Impact on Churn:
    - Security Services are crucial retention tools:
        - Online Security (-0.289) and Tech Support (-0.282) show strong negative correlations with churn
        - These services act as "sticky" features keeping customers engaged
    - Basic Services have less impact:
        - Phone Service and Internet Service alone don't significantly prevent churn
        - Value-added services are more important for retention

3. Pricing and Payment Insights:
    - Monthly Charges (0.193) slightly increases churn risk:
        - Higher bills correlate with increased churn
        - Multiple services (streaming, phone lines) increase monthly charges
    - Paperless Billing (0.192) shows positive correlation with churn:
        - May indicate less engaged or more transient customers

4. Customer Demographics:
    - Senior Citizens (0.151) show slightly higher churn tendency
    - Partners (-0.150) and Dependents (-0.164) reduce churn risk:
        - Family-connected customers are more stable
        - Single customers might need targeted retention strategies

5. Service Bundle Analysis:
    - Strong correlation between complementary services:
        - StreamingTV and StreamingMovies (0.435)
        - Multiple Lines and Monthly Charges (0.434)
    - Suggests successful cross-selling opportunities

#### Strategic Recommendations
1. Immediate Action Items:
    - Promote longer-term contracts with incentives
    - Enhance and actively market security and tech support services
    - Develop special retention programs for customers in first year
2. Customer Segment Focus:
    - Create family-oriented service bundles
    - Develop specific retention strategies for senior citizens
    - Target single customers with engagement programs
3. Service Enhancement:
    - Bundle security features with basic services
    - Offer tech support as part of standard packages
    - Create value-added service combinations
4. Pricing Strategies:
    - Review pricing for high-risk segments
    - Develop loyalty discounts based on tenure
    - Create bundle discounts for complementary services
5. Retention Program Development:
    - Focus on first-year customer experience
    - Implement early warning system for churn risk
    - Create upgrade paths to longer-term contracts
6. Long-term Strategic Focus:
    - Invest in technical support and security features
    - Develop family-oriented service packages
    - Create loyalty programs rewarding tenure

### Box Plots for Monthly Charges

In [15]:
# Create box plot
fig_charges = px.box(
    df,
    x='Churn',
    y='MonthlyCharges',
    title='Monthly Charges Distribution by Churn Status',
    points='outliers',
    width=800,
    height=500
)

# Update layout
fig_charges.update_layout(
    title={
        'text': 'Monthly Charges Distribution by Churn Status',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title="Churn Status",
    yaxis_title="Monthly Charges ($)",
    showlegend=False,
)

# Add mean line
fig_charges.update_traces(boxmean=True)

# Display
fig_charges.show()

# Calculate detailed statistics
stats_by_churn = df.groupby('Churn')['MonthlyCharges'].describe()
print("\nDetailed Statistics of Monthly Charges by Churn Status:")
print(stats_by_churn)

# Additional statistical analysis
print("\nDetailed Analysis:")
for churn in [0, 1]:
    charges = df_encoded[df_encoded['Churn'] == churn]['MonthlyCharges']
    q1 = charges.quantile(0.25)
    q3 = charges.quantile(0.75)
    iqr = q3 - q1
    
    print(f"\nChurn Status: {'Yes' if churn == 1 else 'No'}")
    print(f"Median Monthly Charges: ${charges.median():.2f}")
    print(f"Average Monthly Charges: ${charges.mean():.2f}")
    print(f"Standard Deviation: ${charges.std():.2f}")
    print(f"Interquartile Range: ${iqr:.2f}")
    print(f"Range: ${charges.max() - charges.min():.2f}")
    print(f"Minimum: ${charges.min():.2f}")
    print(f"Maximum: ${charges.max():.2f}")


Detailed Statistics of Monthly Charges by Churn Status:
        count       mean        std    min    25%     50%   75%     max
Churn                                                                  
No     5174.0  61.265124  31.092648  18.25  25.10  64.425  88.4  118.75
Yes    1869.0  74.441332  24.666053  18.85  56.15  79.650  94.2  118.35

Detailed Analysis:

Churn Status: No
Median Monthly Charges: $64.43
Average Monthly Charges: $61.27
Standard Deviation: $31.09
Interquartile Range: $63.30
Range: $100.50
Minimum: $18.25
Maximum: $118.75

Churn Status: Yes
Median Monthly Charges: $79.65
Average Monthly Charges: $74.44
Standard Deviation: $24.67
Interquartile Range: $38.05
Range: $99.50
Minimum: $18.85
Maximum: $118.35



#### Analysis
1. Average Spending Patterns:
    - Churned Customers ($74.44) spend on average $13.17 more than loyal customers ($61.27)
    - This represents a ~21.5% higher average monthly charge for churned customers
    - Clear indication that higher bills increase churn risk
2. Distribution Characteristics:
    - Median Comparison:
        - Loyal customers: $64.43
        - Churned customers: $79.65
        - The $15.22 difference in medians suggests the price sensitivity threshold might be around $70-75
    -   Spread Analysis:
        - Loyal customers show higher variability (std: $31.09) compared to churned customers   (std: $24.67)
        - Wider interquartile range for loyal customers ($63.30) vs churned ($38.05)
        - Suggests more price tolerance among loyal customers
3. Range Analysis:
    - Similar Price Ranges:
        - Loyal: $18.25 - $118.75 (range: $100.50)
        - Churned: $18.85 - $118.35 (range: $99.50)
    - Minimal difference in maximum charges indicates price alone isn't the sole factor

### Churn Rate by Contract Type

In [16]:
# Calculate churn rates by contract type
churn_by_contract = df.groupby('Contract')['Churn'].value_counts(normalize=True).unstack()

# Reset index to make Contract a column
churn_by_contract_reset = churn_by_contract.reset_index()

# Calculate total customers in each contract type
contract_counts = df['Contract'].value_counts()

# Melt the dataframe for proper plotting
churn_by_contract_melted = churn_by_contract_reset.melt(
    id_vars=['Contract'],
    var_name='Churn_Status',
    value_name='Percentage'
)

# Create enhanced bar plot
fig_contract = px.bar(
    churn_by_contract_melted,
    x='Contract',
    y='Percentage',
    color='Churn_Status',
    title='Churn Rate by Contract Type',
    barmode='group',
    width=800,
    height=500,
    # text=churn_by_contract_melted['Percentage'].round(3) * 100
)

# Update layout
fig_contract.update_layout(
    title={
        'text': 'Churn Rate by Contract Type',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title="Contract Type",
    yaxis_title="Percentage (%)",
    yaxis_tickformat=',.1%'
)

# Update traces to show percentages
# fig_contract.update_traces(texttemplate='%{text:.1f}%', textposition='outside')

fig_contract.show()

# Print detailed statistics
print("\nDetailed Analysis of Churn by Contract Type:")
print("\n1. Churn Rates:")
for contract in churn_by_contract.index:
    print(f"\n{contract}:")
    print(f"Retention Rate: {churn_by_contract.loc[contract, 'No']*100:.1f}%")
    print(f"Churn Rate: {churn_by_contract.loc[contract, 'Yes']*100:.1f}%")
    print(f"Total Customers: {contract_counts[contract]}")


Detailed Analysis of Churn by Contract Type:

1. Churn Rates:

Month-to-month:
Retention Rate: 57.3%
Churn Rate: 42.7%
Total Customers: 3875

One year:
Retention Rate: 88.7%
Churn Rate: 11.3%
Total Customers: 1473

Two year:
Retention Rate: 97.2%
Churn Rate: 2.8%
Total Customers: 1695


#### Analysis
1. Contract Distribution Overview:
    - Total Customers: 7,043
    - Month-to-month: 3,875 (55.0%)
    - Two year: 1,695 (24.1%)
    - One year: 1,473 (20.9%)

##### Detailed Analysis by Contract Type: 
1. Month-to-Month Contracts:
    - Highest Risk Segment (42.7% churn)
    - 1,655 customers churned out of 3,875
    - Critical metrics:
        - More than 4 in 10 customers leave
        - Represents majority of customer base (55%)
        - Retention rate of only 57.3%
2. One Year Contracts:
    - Moderate Risk Segment (11.3% churn)
    - 166 customers churned out of 1,473
    - Significant improvements:
        - 31.4% lower churn than month-to-month
        - Strong 88.7% retention rate
        - Shows value of longer commitment
3. Two Year Contracts:
    - Lowest Risk Segment (2.8% churn)
    - Only 47 customers churned out of 1,695
    - Best performance:
        - 97.2% retention rate
        - 8.5% improvement over one-year contracts
        - Demonstrates optimal contract length

### Distribution of Customer Tenure

In [17]:
fig_tenure = px.histogram(
    df,
    x='tenure',
    color='Churn',
    marginal='box',
    title='Customer Tenure Distribution by Churn Status',
    width=1000,  # Set overall figure width
    height=600  # Set overall figure height
)
fig_tenure.show()

# Get statistical summary by churn status
tenure_stats = df.groupby('Churn')['tenure'].describe()
print("Tenure Statistics by Churn Status:")
print(tenure_stats)

# Calculate additional metrics
for churn in ['Yes', 'No']:
    tenure_data = df[df['Churn'] == churn]['tenure']
    print(f"\nAdditional metrics for {churn} churn:")
    print(f"Mode: {tenure_data.mode().values[0]:.1f}")
    print(f"Skewness: {tenure_data.skew():.3f}")
    print(f"Kurtosis: {tenure_data.kurtosis():.3f}")

Tenure Statistics by Churn Status:
        count       mean        std  min   25%   50%   75%   max
Churn                                                           
No     5174.0  37.569965  24.113777  0.0  15.0  38.0  61.0  72.0
Yes    1869.0  17.979133  19.531123  1.0   2.0  10.0  29.0  72.0

Additional metrics for Yes churn:
Mode: 1.0
Skewness: 1.149
Kurtosis: 0.196

Additional metrics for No churn:
Mode: 72.0
Skewness: -0.032
Kurtosis: -1.414


#### Analysis
1. Comparison of Central Tendencies:
    - Loyal Customer (No Churn)
        - Mean tenure: 37.6 months
        - Median tenure: 38.0 months
        - Mode: 72.0 months
        - Nearly symmetric distribution (skewness: -0.032)
    - Churned Customers:
        - Mean tenure: 18.0 months
        - Median tenure: 10.0 months
        - Mode: 1.0 month
        - Highly right-skewed (skewness: 1.149)
2. Distribution Characteristics:
    - Spread Analysis:
        - Retained customers: Higher spread (std: 24.1 months)
        - Churned customers: Lower spread (std: 19.5 months)
        - Both groups max at 72 months
        - Retained minimum: 0 months
        - Churned minimum: 1 month
    - Quartile Analysis:
        - Retained Customers:
            - 25th percentile: 15 months
            - 75th percentile: 61 months
            - IQR: 46 months
        - Churned Customers:
            - 25th percentile: 2 months
            - 75th percentile: 29 months
            - IQR: 27 months
3. Critical Insights:
    - Early Churn Risk:
        - Mode of 1 month for churned customers indicates critical first month
        - 25% of churned customers leave within 2 months
        - 50% leave within 10 months
        - Early intervention window is crucial
    - Loyalty Development:
        - Mode of 72 months for retained customers shows strong loyalty potential
        - Even distribution around mean (37.6 months) for retained customers
        - Negative kurtosis (-1.414) indicates consistent spread across tenure periods

### Churn Rate by Payment Method

In [19]:
payment_churn = df.groupby('PaymentMethod')['Churn'].value_counts(normalize=True).unstack()
fig_payment = px.bar(
    payment_churn,
    title='Churn Rate by Payment Method',
    barmode='group'
)
fig_payment.show()

#### Analysis
1. Electronic Check (Highest Risk Segment):
    - Represents largest customer base (33.58%)
    - Alarmingly high churn rate (45.29%)
    - Highest monthly charges for retained customers ($74.23)
    - Critical Observation: Small price sensitivity ($4.47 difference between churned and retained)
    - Action Required: This segment needs immediate intervention
2. Automatic Payment Methods (Most Stable):
    - **Bank Transfer**:
        - 21.92% customer base
        - Excellent retention (83.29%)
        - $12.83 charge gap between churned/retained
    - **Credit Card**:
        - 21.61% customer base
        - Best retention (84.76%)
        - $12.80 charge gap between churned/retained
    - **Success Pattern**: Both show similar stable patterns
3. Mailed Check (Most Price Sensitive):
    - 22.89% customer base
    - Good retention (80.89%)
    - Lowest average charges ($41.40 retained)
    - Largest price sensitivity ($13.16 difference)
    - Insight: Most price-conscious segment

### Churn Rate by Services

In [ ]:
service_columns = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
                  'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

churn_rates = []
for service in service_columns:
    churn_rate = df.groupby(service)['Churn'].value_counts(normalize=True).unstack()
    churn_rates.append(churn_rate)

# Create subplots for services
fig_services = make_subplots(
    rows=3,
    cols=3,
    subplot_titles=service_columns
)

for idx, (service, churn_rate) in enumerate(zip(service_columns, churn_rates)):
    row = idx // 3 + 1
    col = idx % 3 + 1
    
    fig_services.add_trace(
        go.Bar(x=churn_rate.index, y=churn_rate['Yes'], name=service),
        row=row,
        col=col
    )

fig_services.update_layout(
    height=1000,
    title_text="Churn Rate by Services",
    showlegend=False
)
fig_services.show()

#### Analysis
1. Internet Service (Highest Impact):
    - **Critical Issue**: Fiber optic has 41.89% churn rate vs DSL's 18.96%
    - **Opportunity**: Customers with no internet service have only 7.40% churn
    - **Key Insight**: While fiber optic has highest adoption (43.96%), it's also bleeding customers the most
    - **Action Items**:
        - Investigate fiber optic service quality and pricing
        - Consider DSL as a stable service offering
        - Analyze why no-internet customers are more loyal
2. Security Services (Strong Retention Effect):
    - **OnlineSecurity**:
        - Without: 41.77% churn
        - With: 14.61% churn (27.16% difference)
    - **TechSupport**:
        - Without: 41.64% churn
        - With: 15.17% churn (26.47% difference)
    - **Action Items**:
        - Prioritize these services for upselling
        - Only 28.67% have OnlineSecurity and 29.02% have TechSupport - huge growth potential
3. Backup & Protection Services (Moderate Impact):
    - **OnlineBackup**:
        - Without: 39.93% churn
        - With: 21.53% churn
    - **DeviceProtection**:
        - Without: 39.13% churn
        - With: 22.50% churn
    - **Action Items**:
        - Bundle these services with security offerings
        - Target the ~44% of customers without these services
4. Entertainment Services (Minimal Impact):
    - **StreamingTV & Movies**:
        - Similar patterns: ~30% churn with service, ~33% without
        - Good adoption rates: ~38% for both services
    - **Action Items**:
        - Not primary focus for churn reduction
        - Consider bundling with high-impact services
5. Phone Services (Low Impact):
    - **Basic Phone Service**:
        - High adoption (90.32%) but minimal churn difference
    - **Multiple Lines**:
        - Slightly higher churn with multiple lines (28.61% vs 25.04%)
    - **Action Items**:
        - Focus on upselling internet and security services instead

### Treemap of Churn Rate by Services

In [20]:
# Create a list to store the data for treemap
treemap_data = []

for service in service_columns:
    # Calculate churn rates for each service category
    churn_by_service = df.groupby([service, 'Churn']).size().reset_index(name='count')
    total_by_service = churn_by_service.groupby(service)['count'].sum().reset_index()
    
    # Merge total counts with churn counts
    churn_by_service = churn_by_service.merge(total_by_service, on=service, suffixes=('', '_total'))
    churn_by_service['percentage'] = (churn_by_service['count'] / churn_by_service['count_total'] * 100).round(2)
    
    # Add to treemap data
    for _, row in churn_by_service.iterrows():
        treemap_data.append({
            'Service': 'Services',  # Root level
            'Type': service,        # Service type
            'Category': f"{row[service]}",  # Service category
            'Status': row['Churn'],  # Churn status
            'Count': row['count'],
            'Percentage': row['percentage']
        })

# Create DataFrame for treemap
df_treemap = pd.DataFrame(treemap_data)

# Create treemap
fig = px.treemap(
    df_treemap,
    path=['Service', 'Type', 'Category', 'Status'],
    values='Count',
    color='Percentage',
    color_continuous_scale='RdYlGn',  # Red for high churn, green for low churn
    title='Service Usage and Churn Rate Distribution',
    custom_data=['Percentage']
)

# Update layout
fig.update_layout(
    width=1200,
    height=800,
    title={
        'text': 'Service Usage and Churn Rate Distribution',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 20}
    }
)

# Update hover template
fig.update_traces(
    hovertemplate="<b>%{label}</b><br>" +
                  "Count: %{value}<br>" +
                  "Churn Rate: %{customdata[0]:.1f}%<extra></extra>"
)

# Show the plot
fig.show()

#### Analysis
1. Core Internet Services (Highest Impact):
    - Fiber Optic: 41.89% churn (43.96% base)
    - DSL: 18.96% churn (34.37% base)
    - No Internet: 7.40% churn (21.67% base)
    - Impact Range: 34.49% (Fiber vs No Internet)
    - Critical Finding: Internet service type is the strongest predictor of churn
2. Security & Support Tier (High Impact):
    - Online Security:
        - Without: 41.77% churn (49.67% base)
        - With: 14.61% churn (28.67% base)
        - Impact: 27.16% reduction

    - Tech Support:
        - Without: 41.64% churn (49.31% base)
        - With: 15.17% churn (29.02% base)
        - Impact: 26.47% reduction
    
    - Key Insight: Security and support services have nearly identical churn reduction effects
3. Protection Services Tier (Medium Impact):
    - Online Backup:
        - Without: 39.93% churn (43.84% base)
        - With: 21.53% churn (34.49% base)
        - Impact: 18.40% reduction

    - Device Protection:
        - Without: 39.13% churn (43.94% base)
        - With: 22.50% churn (34.39% base)
        - Impact: 16.63% reduction
    
    - Pattern: Similar adoption rates and churn reduction effects
4. Entertainment Services (Low Impact):
    - Streaming TV:
        - Without: 33.52% churn (39.90% base)
        - With: 30.07% churn (38.44% base)
        - Impact: 3.45% reduction

    - Streaming Movies:
        - Without: 33.68% churn (39.54% base)
        - With: 29.94% churn (38.79% base)
        - Impact: 3.74% reduction
    
    - Note: Minimal impact on churn reduction
5. Phone Services (Minimal Impact):
    - Basic Phone:
        - Without: 24.93% churn (9.68% base)
        - With: 26.71% churn (90.32% base)
        - Impact: -1.78% (slightly increases churn)

    - Multiple Lines:
        - Without: 25.04% churn (48.13% base)
        - With: 28.61% churn (42.18% base)
        - Impact: -3.57% (increases churn)

# Level 3

## Data Preparation

In [21]:
# Convert numeric columns and handle missing values
df_encoded['TotalCharges'] = pd.to_numeric(df_encoded['TotalCharges'], errors='coerce')
df_encoded['MonthlyCharges'] = pd.to_numeric(df_encoded['MonthlyCharges'], errors='coerce')
df_encoded['tenure'] = pd.to_numeric(df_encoded['tenure'], errors='coerce')

# Fill NaN values with mean for numeric columns without using inplace
numeric_columns = ['TotalCharges', 'MonthlyCharges', 'tenure']
df_encoded[numeric_columns] = df_encoded[numeric_columns].fillna(df_encoded[numeric_columns].mean())

# Drop customerID if it exists
if 'customerID' in df_encoded.columns:
    df_encoded = df_encoded.drop('customerID', axis=1)

# Separate features and target
X = df_encoded.drop('Churn', axis=1)
y = df_encoded['Churn']

# Print data info before scaling
print("Features shape:", X.shape)
print("\nFeature columns:", list(X.columns))
print("\nChecking for NaN values before scaling:", X.isna().sum().sum())

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Print scaling results
print("\nAfter scaling:")
print("X_scaled shape:", X_scaled.shape)
print("Contains NaN:", np.any(np.isnan(X_scaled)))
print("Contains inf:", np.any(np.isinf(X_scaled)))

Features shape: (7043, 19)

Feature columns: ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges']

Checking for NaN values before scaling: 0

After scaling:
X_scaled shape: (7043, 19)
Contains NaN: False
Contains inf: False


## Split the dataset

In [22]:
# Split the enhanced dataset
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

## Build the model

In [ ]:
tf_model = models.Sequential([
    # Input layer
    layers.Input(shape=(X_train.shape[1],)),
    
    # First dense block
    layers.Dense(512, kernel_regularizer=l2(0.0001),
                kernel_initializer='he_uniform'),
    layers.LeakyReLU(negative_slope=0.02),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    # Second dense block
    layers.Dense(256, kernel_regularizer=l2(0.0001),
                kernel_initializer='he_uniform'),
    layers.LeakyReLU(negative_slope=0.02),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    
    # Third dense block
    layers.Dense(128, kernel_regularizer=l2(0.0001),
                kernel_initializer='he_uniform'),
    layers.LeakyReLU(negative_slope=0.02),
    layers.BatchNormalization(),
    layers.Dropout(0.2),

    # Fourth dense block
    layers.Dense(64, kernel_regularizer=l2(0.0001),
                kernel_initializer='he_uniform'),
    layers.LeakyReLU(negative_slope=0.02),
    layers.BatchNormalization(),
    layers.Dropout(0.15),
    
    # Fifth dense block
    layers.Dense(32, kernel_regularizer=l2(0.0001),
                kernel_initializer='he_uniform'),
    layers.LeakyReLU(negative_slope=0.02),
    layers.BatchNormalization(),
    layers.Dropout(0.1),
    
    # Output layer
    layers.Dense(1, activation='sigmoid')
])

# Option 1: Using constant learning rate with ReduceLROnPlateau
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
tf_model.compile(optimizer=optimizer,
               loss='binary_crossentropy',
               metrics=['accuracy'])

# Add ModelCheckpoint callback
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    '_best_model.keras',  # Path to save the model
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1,
    save_weights_only=False  # Save the entire model
)

# Add other callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.2,
    patience=10,
    min_lr=0.00001,
    verbose=1  # Added verbose to see when learning rate changes
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=15,
    restore_best_weights=True,
    verbose=1  # Added verbose to see when early stopping occurs
)

# Train the model with all callbacks
history = tf_model.fit(X_train, y_train,
                    epochs=150,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping, reduce_lr, checkpoint],
                    verbose=1)

# Load the best model
tf_model.save('_final_model.keras')

Epoch 1/150


I0000 00:00:1733110031.004210   92850 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733110031.033974   92850 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733110031.034185   92850 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733110031.035235   92850 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

 94/155 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6286 - loss: 0.9380

I0000 00:00:1733110036.753673   92908 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6517 - loss: 0.9018
Epoch 1: val_accuracy improved from -inf to 0.78883, saving model to best_model.keras
155/155 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.6520 - loss: 0.9014 - val_accuracy: 0.7888 - val_loss: 0.6826 - learning_rate: 0.0010
Epoch 2/150
127/155 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7567 - loss: 0.7158
Epoch 2: val_accuracy improved from 0.78883 to 0.79545, saving model to best_model.keras
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7572 - loss: 0.7135 - val_accuracy: 0.7955 - val_loss: 0.6333 - learning_rate: 0.0010
Epoch 3/150
128/155 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7719 - loss: 0.6807
Epoch 3: val_accuracy improved from 0.79545 to 0.80492, saving model to best_model.keras
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7718 - loss: 0.6795 - val_accuracy: 0.8049 - val_loss: 0.6234 - learning_rate: 0.0010
Epoch 4/150
125/155 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accurac

## Pickle history

In [ ]:
with open('./_historymodel.pkl', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

## Visualize

In [25]:
# Calculate and plot learning curve
epochs = list(range(1, len(history.history['accuracy']) + 1))  # Convert range to list
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Find best epoch
best_epoch = np.argmax(history.history['val_accuracy'])

# Create subplots for accuracy and loss
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Model Accuracy', 'Model Loss'))

# Add accuracy traces
fig.add_trace(
    go.Scatter(y=history.history['accuracy'], name="Training Accuracy",
               line=dict(color='blue')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(y=history.history['val_accuracy'], name="Validation Accuracy",
               line=dict(color='red')),
    row=1, col=1
)

# Add loss traces
fig.add_trace(
    go.Scatter(y=history.history['loss'], name="Training Loss",
               line=dict(color='blue')),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(y=history.history['val_loss'], name="Validation Loss",
               line=dict(color='red')),
    row=1, col=2
)

# Update layout
fig.update_layout(height=500, width=1000, title_text="Training Metrics")
fig.update_xaxes(title_text="Epoch", row=1, col=1)
fig.update_xaxes(title_text="Epoch", row=1, col=2)
fig.update_yaxes(title_text="Accuracy", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=1, col=2)

fig.show()

# Generate predictions and create confusion matrix
y_pred = tf_model.predict(X_val)
y_pred_classes = (y_pred > 0.5).astype(int)

# Calculate confusion matrix
cm = confusion_matrix(y_val, y_pred_classes)

# Create confusion matrix heatmap
fig_cm = go.Figure(data=go.Heatmap(
    z=cm,
    x=['Predicted No Churn', 'Predicted Churn'],
    y=['Actual No Churn', 'Actual Churn'],
    text=cm,
    texttemplate="%{text}",
    textfont={"size": 16},
    colorscale='RdYlBu'
))

fig_cm.update_layout(
    title='Confusion Matrix',
    xaxis_title='Predicted Label',
    yaxis_title='True Label',
    width=600,
    height=500
)

fig_cm.show()

# Print classification report
print("\nClassification Report:")
print(classification_report(y_val, y_pred_classes))

# Calculate ROC curve
fpr, tpr, _ = roc_curve(y_val, y_pred)
roc_auc = auc(fpr, tpr)

# Create ROC curve plot
fig_roc = go.Figure()

fig_roc.add_trace(
    go.Scatter(x=fpr, y=tpr, 
               name=f'ROC curve (AUC = {roc_auc:.3f})',
               line=dict(color='#2ECC71', width=2))
)

fig_roc.add_trace(
    go.Scatter(x=[0, 1], y=[0, 1],
               name='Random Classifier',
               line=dict(color='#E74C3C', width=2, dash='dash'))
)

fig_roc.update_layout(
    title='Receiver Operating Characteristic (ROC) Curve',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    template='plotly_white',
    width=800,
    height=500,
    showlegend=True
)

fig_roc.show()

# Print summary of best performance
print("\nModel Performance Summary:")
print(f"Best Validation Accuracy: {max(history.history['val_accuracy']):.4f}")
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")
print(f"\nTotal Epochs Trained: {len(history.history['accuracy'])}")
print(f"Best Epoch: {best_epoch + 1}")  # Adding 1 because epochs are 1-indexed

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step



Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.91      0.88       778
           1       0.69      0.58      0.63       278

    accuracy                           0.82      1056
   macro avg       0.77      0.74      0.75      1056
weighted avg       0.81      0.82      0.81      1056




Model Performance Summary:
Best Validation Accuracy: 0.8201
Final Validation Accuracy: 0.8097
ROC AUC Score: 0.8514

Total Epochs Trained: 49
Best Epoch: 34


In [26]:
# Load both models
best_model = tf.keras.models.load_model('best_model.keras')
final_model = tf.keras.models.load_model('final_model.keras')

# Get predictions from both models
best_pred = best_model.predict(X_val)
final_pred = final_model.predict(X_val)

# Convert probabilities to class predictions
best_pred_classes = (best_pred > 0.5).astype(int)
final_pred_classes = (final_pred > 0.5).astype(int)

# Calculate confusion matrices
best_cm = confusion_matrix(y_val, best_pred_classes)
final_cm = confusion_matrix(y_val, final_pred_classes)

# Calculate ROC curves
best_fpr, best_tpr, _ = roc_curve(y_val, best_pred)
final_fpr, final_tpr, _ = roc_curve(y_val, final_pred)
best_auc = auc(best_fpr, best_tpr)
final_auc = auc(final_fpr, final_tpr)

# Create subplot figure
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Best Model Confusion Matrix', 
                                  'Final Model Confusion Matrix'))

# Add confusion matrix heatmaps
fig.add_trace(
    go.Heatmap(
        z=best_cm,
        x=['Predicted No Churn', 'Predicted Churn'],
        y=['Actual No Churn', 'Actual Churn'],
        text=best_cm,
        texttemplate="%{text}",
        textfont={"size": 16},
        colorscale='RdYlBu',
        showscale=False
    ),
    row=1, col=1
)

fig.add_trace(
    go.Heatmap(
        z=final_cm,
        x=['Predicted No Churn', 'Predicted Churn'],
        y=['Actual No Churn', 'Actual Churn'],
        text=final_cm,
        texttemplate="%{text}",
        textfont={"size": 16},
        colorscale='RdYlBu',
        showscale=False
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(
    title='Confusion Matrix Comparison',
    width=1200,
    height=500
)

fig.show()

# Create ROC curve comparison
fig_roc = go.Figure()

# Add ROC curves for both models
fig_roc.add_trace(
    go.Scatter(x=best_fpr, y=best_tpr,
               name=f'Best Model (AUC = {best_auc:.3f})',
               line=dict(color='#2ECC71', width=2))
)

fig_roc.add_trace(
    go.Scatter(x=final_fpr, y=final_tpr,
               name=f'Final Model (AUC = {final_auc:.3f})',
               line=dict(color='#E74C3C', width=2))
)

fig_roc.add_trace(
    go.Scatter(x=[0, 1], y=[0, 1],
               name='Random Classifier',
               line=dict(color='gray', width=2, dash='dash'))
)

fig_roc.update_layout(
    title='ROC Curve Comparison',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    template='plotly_white',
    width=800,
    height=500,
    showlegend=True
)

fig_roc.show()

# Print classification reports
print("\nBest Model Classification Report:")
print(classification_report(y_val, best_pred_classes))

print("\nFinal Model Classification Report:")
print(classification_report(y_val, final_pred_classes))

# Print accuracy comparison
best_acc = best_model.evaluate(X_val, y_val, verbose=0)[1]
final_acc = final_model.evaluate(X_val, y_val, verbose=0)[1]

print("\nModel Accuracy Comparison:")
print(f"Best Model Validation Accuracy: {best_acc:.4f}")
print(f"Final Model Validation Accuracy: {final_acc:.4f}")
print(f"Difference (Best - Final): {(best_acc - final_acc):.4f}")

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 769us/step



Best Model Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.91      0.88       778
           1       0.69      0.58      0.63       278

    accuracy                           0.82      1056
   macro avg       0.77      0.74      0.75      1056
weighted avg       0.81      0.82      0.81      1056


Final Model Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.91      0.88       778
           1       0.69      0.58      0.63       278

    accuracy                           0.82      1056
   macro avg       0.77      0.74      0.75      1056
weighted avg       0.81      0.82      0.81      1056


Model Accuracy Comparison:
Best Model Validation Accuracy: 0.8201
Final Model Validation Accuracy: 0.8201
Difference (Best - Final): 0.0000


In [28]:
def calculate_feature_importance(model, X, y, feature_names, n_repeats=10):
    # Convert X to numpy array if it's a DataFrame
    X_array = X.values if hasattr(X, 'values') else X
    
    baseline_score = model.evaluate(X_array, y, verbose=0)[1]  # Get baseline accuracy
    importance_scores = []
    print(f"Baseline accuracy: {baseline_score:.4f}")  # Add this for debugging
    
    # For each feature
    for i, feature in enumerate(feature_names):
        scores = []
        
        # Repeat n times
        for _ in range(n_repeats):
            # Create a copy of the input data
            X_permuted = X_array.copy()
            # Permute the feature
            X_permuted[:, i] = np.random.permutation(X_permuted[:, i])
            # Calculate new score
            new_score = model.evaluate(X_permuted, y, verbose=0)[1]
            # Calculate importance as decrease in performance
            importance = baseline_score - new_score
            scores.append(importance)
            
            # Add this for debugging
            if _ == 0:  # Print only for first iteration
                print(f"Feature {feature}: original score = {baseline_score:.4f}, permuted score = {new_score:.4f}, importance = {importance:.4f}")
        
        # Average importance over repeats
        mean_importance = np.mean(scores)
        importance_scores.append(mean_importance)
    
    return importance_scores

# Get feature names if X is a DataFrame, otherwise use the provided feature_names
feature_names = X.columns.tolist() if hasattr(X, 'columns') else feature_names

# Calculate feature importance with fewer repeats first to test
importance_scores = calculate_feature_importance(tf_model, X_test, y_test, feature_names, n_repeats=5)

# Create DataFrame with feature importance and sort
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importance_scores
})
importance_df = importance_df.sort_values('importance')

# Determine colors based on importance
colors = ['red' if x < 0 else 'green' for x in importance_df['importance']]

# Create the diverging bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    y=importance_df['feature'],
    x=importance_df['importance'],
    orientation='h',
    marker_color=colors,
    marker_line_color='white',
    marker_line_width=0.5,
    opacity=0.7
))

fig.update_layout(
    title={
        'text': 'Feature Importance for Churn Prediction (TensorFlow Model)',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 20}
    },
    xaxis_title='Impact on Model Accuracy (+ means more important)',
    yaxis_title='Features',
    width=1000,
    height=800,
    xaxis=dict(
        zeroline=True,
        zerolinewidth=2,
        zerolinecolor='black',
        gridcolor='lightgrey',
        griddash='dash',
        dtick=0.05  # Adjust based on your importance values
    ),
    yaxis=dict(
        tickfont=dict(size=12)
    ),
    plot_bgcolor='white'
)

# Add a vertical line at x=0
fig.add_vline(x=0, line_width=1, line_color="black")

# Display
fig.show()

Baseline accuracy: 0.7966
Feature gender: original score = 0.7966, permuted score = 0.7928, importance = 0.0038
Feature SeniorCitizen: original score = 0.7966, permuted score = 0.7966, importance = 0.0000
Feature Partner: original score = 0.7966, permuted score = 0.7975, importance = -0.0009
Feature Dependents: original score = 0.7966, permuted score = 0.7938, importance = 0.0028
Feature tenure: original score = 0.7966, permuted score = 0.7531, importance = 0.0435
Feature PhoneService: original score = 0.7966, permuted score = 0.7919, importance = 0.0047
Feature MultipleLines: original score = 0.7966, permuted score = 0.7900, importance = 0.0066
Feature InternetService: original score = 0.7966, permuted score = 0.7843, importance = 0.0123
Feature OnlineSecurity: original score = 0.7966, permuted score = 0.7919, importance = 0.0047
Feature OnlineBackup: original score = 0.7966, permuted score = 0.8023, importance = -0.0057
Feature DeviceProtection: original score = 0.7966, permuted scor